In [8]:
import json
import time
import os
import praw

# Reddit API credentials
reddit = praw.Reddit(
    client_id="bgpEcNkIWPdnxbMZ2sO8uw",
    client_secret="9dNxl_O_ZAjqFCwDEUGISoxBiDr9yQ",
    user_agent="PythonAPI",
)

# Configuration
SUBREDDIT_NAME = "lonely"  # Change to any subreddit
OUTPUT_FILE = "C:/Users/w065pxg/Desktop/ML/Tanvi Banerjee 2024 MI/Final Project/StudentResearch/RedditData.json"

PULL_LIMIT = 250  # Number of posts to pull
KEYWORD = ""  # Leave empty to fetch all posts

# Load existing data if available
flair_data = {}
meta_data = {}
pulled_ids = set()
total_size = 0

if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            flair_data = data.get('flair', {})
            meta_data = data.get('meta', {})
            pulled_ids = set(meta_data.keys())
            total_size = len(pulled_ids)
            print(f"Loaded {total_size} posts from file.")
        except json.JSONDecodeError:
            print("Existing file is empty or corrupted. Starting fresh.")

# Connect to subreddit
subreddit = reddit.subreddit(SUBREDDIT_NAME)

# Start fetching posts
count_posts = 0
count_duplicates = 0
count_skipped_no_comments = 0

try:
    print(f"Fetching {PULL_LIMIT} posts from r/{SUBREDDIT_NAME}...")

    # Use `.new(limit=PULL_LIMIT)` to get the most recent posts
    for post in subreddit.new(limit=PULL_LIMIT):
        if count_posts >= PULL_LIMIT:
            break

        if post.id in pulled_ids:
            print(f"⚠️ Duplicate skipped: {post.id}")
            count_duplicates += 1
            continue

        if post.num_comments == 0:
            print(f"⚠️ Skipping post with no comments: {post.id}")
            count_skipped_no_comments += 1
            continue

        print(f"✔️ {count_posts+1}. {post.title}")
        flair = post.link_flair_text or "No Flair"
        post_info = {
            'title': post.title,
            'author': str(post.author),
            'score': post.score,
            'text_body': post.selftext if post.is_self else post.url,
            'num_comments': post.num_comments,
            'comments': {}
        }

        # Fetch and expand all hidden comments
        post.comment_sort = "new"  # Sort by newest comments
        post.comments.replace_more(limit=None)

        for comment in post.comments.list():
            if comment.body == '[deleted]':
                continue
            post_info['comments'][comment.id] = {
                'comment': comment.body,
                'score': comment.score,
                'parentId': comment.parent_id.split('_')[1] if '_' in comment.parent_id else None
            }    

        # Save post data
        if flair not in flair_data:
            flair_data[flair] = {}
        flair_data[flair][post.id] = post_info
        meta_data[post.id] = {'flair': flair, 'keywords': [KEYWORD] if KEYWORD else []}

        # Add post ID to pulled set
        pulled_ids.add(post.id)

        time.sleep(2)  # Respect Reddit's rate limits
        count_posts += 1

except Exception as e:
    print("🚨 An error occurred, saving data before exiting.")
    print(e)

# Save the data
output_data = {'meta': meta_data, 'flair': flair_data}
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)

# Print Summary
print(f"\n✅ Scraping Complete!")
print(f"✔️ New Posts Pulled: {count_posts}")
print(f"✔️ Total Posts Stored: {total_size + count_posts}")
print(f"⚠️ Duplicates Skipped: {count_duplicates}")
print(f"⚠️ Skipped (No Comments): {count_skipped_no_comments}")


Loaded 1 posts from file.
Fetching 250 posts from r/lonely...
⚠️ Skipping post with no comments: 1j5zo8t
⚠️ Duplicate skipped: 1j5zlrl
⚠️ Skipping post with no comments: 1j5z3xw
⚠️ Skipping post with no comments: 1j5ypkx
✔️ 1. I wish I was special to someone
✔️ 2. I'm tired of feeling alone
✔️ 3. Finding myself so alone into my late 30s is life over?
✔️ 4. I hate my life
✔️ 5. I almost cried because of Uber Eats
✔️ 6. This isn’t the way I want to spend my life.
✔️ 7. I’m tired of being alone
✔️ 8. I don't want anything, I just want someone who cares
✔️ 9. no friends for almost 7 years now
✔️ 10. tired
✔️ 11. Brain Vomit
✔️ 12. can someone explain the concept of “friends” to me?
⚠️ Skipping post with no comments: 1j5uohn
✔️ 13. Podcasts help. I can just walk around town and listen to some big long interviews.
✔️ 14. Does drinking from time to time help with loneliness
✔️ 15. I feel so alone
✔️ 16. Social media
✔️ 17. 3 months indoors
✔️ 18. Finally caved and started talking to robots
✔️